In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import networkx as nx
import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout
from networkx.drawing.nx_agraph import to_agraph 
import matplotlib.pyplot as plt
from pylab import hist
import random
from collections import Counter
import operator
import itertools
import cPickle as pickle
import warnings
warnings.filterwarnings("ignore")

import gzip
import matplotlib.dates as mdates

In [2]:
#remove scientific notation
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [3]:
#pull as CSV files together in one df
df = pd.concat(map(pd.read_csv, ['rec_data1.csv']))

In [4]:
#pull a CSV files of onehot encoding of productIds
des = pd.concat(map(pd.read_csv, ['ProdDes.csv']))

In [5]:
des.info()
des.describe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 2 columns):
prod_code     144 non-null int64
product_id    144 non-null object
dtypes: int64(1), object(1)
memory usage: 2.3+ KB


<bound method DataFrame.describe of      prod_code                 product_id
0            0                    aa-2001
1            1                    aa-2102
2            2     aluminum-travel-mug-14
3            3             artprint-11x14
4            4  artprint-11x14-horizontal
5            5             artprint-16x20
6            6  artprint-16x20-horizontal
7            7              artprint-8x10
8            8   artprint-8x10-horizontal
9            9                 bella-1005
10          10                 bella-3200
11          11                 bella-4000
12          12                 bella-6004
13          13                 bella-7501
14          14                 bella-8800
15          15                 bella-8816
16          16               canvas-11x14
17          17    canvas-11x14-horizontal
18          18               canvas-12x12
19          19               canvas-16x16
20          20               canvas-16x20
21          21    canvas-16x20-horizonta

In [6]:
#obtain info on dataset 
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 28 columns):
order_date           240000 non-null object
seller_id            240000 non-null int64
store_name           240000 non-null object
customer_key         240000 non-null object
order_id             240000 non-null object
locked_units         240000 non-null object
campaign_tag         240000 non-null object
campaing_name        240000 non-null object
front_art_key        182751 non-null object
back_art_key         75204 non-null object
super_category       239998 non-null object
category             239998 non-null object
sku                  240000 non-null object
product_id           240000 non-null object
color                240000 non-null object
size                 240000 non-null object
qty                  240000 non-null int64
item_index           240000 non-null int64
items_in_order       240000 non-null int64
line_item_revenue    240000 non-null object
partner_cost  

,seller_id,qty,item_index,items_in_order
count,240000.00,240000.00,240000.00,240000.00
mean,22.00,1.04,1.24,1.49
std,0.00,0.27,0.66,1.01
min,22.00,0.00,1.00,1.00
25%,22.00,1.00,1.00,1.00
50%,22.00,1.00,1.00,1.00
75%,22.00,1.00,1.00,2.00
max,22.00,33.00,16.00,16.00


In [7]:
#change object to float
df['line_item_revenue'] = (df['line_item_revenue'].str.replace(r'[^-+\d.]', '').astype(float))

In [8]:
#change object to float
df['partner_cost'] = (df['partner_cost'].str.replace(r'[^-+\d.]', '').astype(float))

In [9]:
#change object to float
df['vendor_cost'] = (df['vendor_cost'].str.replace(r'[^-+\d.]', '').astype(float))

In [10]:
#ensure float changes took effect
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 28 columns):
order_date           240000 non-null object
seller_id            240000 non-null int64
store_name           240000 non-null object
customer_key         240000 non-null object
order_id             240000 non-null object
locked_units         240000 non-null object
campaign_tag         240000 non-null object
campaing_name        240000 non-null object
front_art_key        182751 non-null object
back_art_key         75204 non-null object
super_category       239998 non-null object
category             239998 non-null object
sku                  240000 non-null object
product_id           240000 non-null object
color                240000 non-null object
size                 240000 non-null object
qty                  240000 non-null int64
item_index           240000 non-null int64
items_in_order       240000 non-null int64
line_item_revenue    240000 non-null float64
partner_cost 

,seller_id,qty,item_index,items_in_order,line_item_revenue,partner_cost,vendor_cost
count,240000.00,240000.00,240000.00,240000.00,240000.00,239997.00,240000.00
mean,22.00,1.04,1.24,1.49,38.77,13.35,7.76
std,0.00,0.27,0.66,1.01,18.42,9.43,6.52
min,22.00,0.00,1.00,1.00,0.00,2.35,1.50
25%,22.00,1.00,1.00,1.00,25.95,6.15,2.72
50%,22.00,1.00,1.00,1.00,32.95,11.75,6.03
75%,22.00,1.00,1.00,2.00,45.95,17.25,12.10
max,22.00,33.00,16.00,16.00,1582.35,569.25,55.93


In [11]:
#Create new column for seller margin
df['Seller_Margin'] = df['line_item_revenue'] - df['partner_cost']

In [12]:
#Create new column for GearLaunch margin
df['GL_Margin'] = df['partner_cost'] - (df['qty'] * df['vendor_cost']) 

In [13]:
#ensure new columns are added
df.tail()

,order_date,seller_id,store_name,customer_key,order_id,locked_units,campaign_tag,campaing_name,front_art_key,back_art_key,...,partner_cost,vendor_cost,vendor,ship_state,ship_city,ship_zip,ship_country,region,Seller_Margin,GL_Margin
239995,2018-12-26,22,bbbTee,ahBzfmdlYXJsYXVuY2gtaHVichILEghDdXN0b21lchjmu6...,OR_87TCVRB,N,"['basketball', 'linh', 'storefront', 'trang']",Believe in yourself,ahBzfmdlYXJsYXVuY2gtaHVicg0LEgNBcnQYqJnSrSAM,NaN,...,13.25,8.69,CIRCLE_GRAPHICS,TX,Fredericksburg,78624,US,Americas,22.70,4.56
239996,2018-12-26,22,bbbTee,ahBzfmdlYXJsYXVuY2gtaHVichILEghDdXN0b21lchjOlt...,OR_87F7H1K,N,"['giang', 'hoa', 'volleyball']",God Sent Me Rebellious Volleyball Daughter,ahBzfmdlYXJsYXVuY2gtaHVicg0LEgNBcnQYvayt7CAM,NaN,...,6.15,1.54,DREAM_JUNCTION,NE,Omaha,68137,US,Americas,18.80,4.61
239997,2018-12-26,22,bbbTee,ahBzfmdlYXJsYXVuY2gtaHVichILEghDdXN0b21lchjt4J...,OR_87TA3TZ,N,"['linh', 'soccer', 'storefront', 'trang']",Believe in yourself,ahBzfmdlYXJsYXVuY2gtaHVicg0LEgNBcnQY-tez3B0M,NaN,...,9.25,6.37,CIRCLE_GRAPHICS,NY,Dix Hills,11746,US,Americas,22.70,2.88
239998,2018-12-26,22,bbbTee,ahBzfmdlYXJsYXVuY2gtaHVichILEghDdXN0b21lchjv3Y...,OR_88AG0BR,N,"['basketball', 'giang', 'linh']",I'm Not Afraid To Foul Out So Don't Even Try Me,ahBzfmdlYXJsYXVuY2gtaHVicg0LEgNBcnQYopvtqCAM,NaN,...,6.15,1.77,DREAM_JUNCTION,TX,Frisco,75034,US,Americas,18.80,4.38
239999,2018-12-26,22,bbbTee,ahBzfmdlYXJsYXVuY2gtaHVichILEghDdXN0b21lchjXiv...,OR_87TBN7K,N,"['linh', 'soccer', 'storefront', 'trang']",Soccer Mom,NaN,ahBzfmdlYXJsYXVuY2gtaHVicg0LEgNBcnQYg52i6B0M,...,8.35,3.64,DREAM_JUNCTION,NC,Wilmington,28409,US,Americas,17.60,4.71


In [14]:
#remove records with margin <0

df = df[df.Seller_Margin > 0]

In [17]:
#remove records with margin <0

df = df[df.GL_Margin > 0]

In [15]:
#ensure new columns make sense
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239938 entries, 0 to 239999
Data columns (total 30 columns):
order_date           239938 non-null object
seller_id            239938 non-null int64
store_name           239938 non-null object
customer_key         239938 non-null object
order_id             239938 non-null object
locked_units         239938 non-null object
campaign_tag         239938 non-null object
campaing_name        239938 non-null object
front_art_key        182690 non-null object
back_art_key         75181 non-null object
super_category       239936 non-null object
category             239936 non-null object
sku                  239938 non-null object
product_id           239938 non-null object
color                239938 non-null object
size                 239938 non-null object
qty                  239938 non-null int64
item_index           239938 non-null int64
items_in_order       239938 non-null int64
line_item_revenue    239938 non-null float64
partner_cost 

,seller_id,qty,item_index,items_in_order,line_item_revenue,partner_cost,vendor_cost,Seller_Margin,GL_Margin
count,239938.00,239938.00,239938.00,239938.00,239938.00,239938.00,239938.00,239938.00,239938.00
mean,22.00,1.04,1.24,1.49,38.78,13.35,7.76,25.43,5.31
std,0.00,0.27,0.66,1.01,18.42,9.43,6.52,10.18,2.88
min,22.00,1.00,1.00,1.00,5.45,2.35,1.50,0.30,-8.74
25%,22.00,1.00,1.00,1.00,25.95,6.15,2.72,18.80,4.38
50%,22.00,1.00,1.00,1.00,32.95,11.75,6.03,22.70,4.64
75%,22.00,1.00,1.00,2.00,45.95,17.25,12.10,30.70,5.82
max,22.00,33.00,16.00,16.00,1582.35,569.25,49.38,1013.10,161.76


## LIGHTFM

In [16]:
from lightfm import LightFM

In [17]:
#sample code from https://github.com/aayushmnit/cookbook/blob/master/notebooks/recsys/Recommender.ipynb

import pandas as pd
import numpy as np
from scipy import sparse
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity

def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    '''
    Function to create an interaction matrix dataframe from transactional type interactions
    Required Input -
        - df = Pandas DataFrame containing user-item interactions
        - user_col = column name containing user's identifier
        - item_col = column name containing item's identifier
        - rating col = column name containing user feedback on interaction with a given item
        - norm (optional) = True if a normalization of ratings is needed
        - threshold (required if norm = True) = value above which the rating is favorable
    Expected output - 
        - Pandas dataframe with user-item interactions ready to be fed in a recommendation algorithm
    '''
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

def create_user_dict(interactions):
    '''
    Function to create a user dictionary based on their index and number in interaction dataset
    Required Input - 
        interactions - dataset create by create_interaction_matrix
    Expected Output -
        user_dict - Dictionary type output containing interaction_index as key and user_id as value
    '''
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict
    
def create_item_dict(df,id_col,name_col):
    '''
    Function to create an item dictionary based on their item_id and item name
    Required Input - 
        - df = Pandas dataframe with Item information
        - id_col = Column name containing unique identifier for an item
        - name_col = Column name containing name of the item
    Expected Output -
        item_dict = Dictionary type output containing item_id as key and item_name as value
    '''
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30,n_jobs = 4):
    '''
    Function to run matrix-factorization algorithm
    Required Input -
        - interactions = dataset create by create_interaction_matrix
        - n_components = number of embeddings you want to create to define Item and user
        - loss = loss function other options are logistic, brp
        - epoch = number of epochs to run 
        - n_jobs = number of cores used for execution 
    Expected Output  -
        Model - Trained model
    '''
    x = sparse.csr_matrix(interactions.values)
    model = LightFM(no_components= n_components, loss=loss,k=k)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    return model

def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True):
    '''
    Function to produce user recommendations
    Required Input - 
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - user_id = user ID for which we need to generate recommendation
        - user_dict = Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - threshold = value above which the rating is favorable in new interaction matrix
        - nrec_items = Number of output recommendation needed
    Expected Output - 
        - Prints list of items the given user has already bought
        - Prints list of N recommended items  which user hopefully will be interested in
    '''
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index) \
								 .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Products Bought:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Products:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list
    

def sample_recommendation_item(model,interactions,item_id,user_dict,item_dict,number_of_user):
    '''
    Funnction to produce a list of top N interested users for a given item
    Required Input -
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - item_id = item ID for which we need to generate recommended users
        - user_dict =  Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - number_of_user = Number of users needed as an output
    Expected Output -
        - user_list = List of recommended users 
    '''
    n_users, n_items = interactions.shape
    x = np.array(interactions.columns)
    scores = pd.Series(model.predict(np.arange(n_users), np.repeat(x.searchsorted(item_id),n_users)))
    user_list = list(interactions.index[scores.sort_values(ascending=False).head(number_of_user).index])
    return user_list 


def create_item_emdedding_distance_matrix(model,interactions):
    '''
    Function to create item-item distance embedding matrix
    Required Input -
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
    Expected Output -
        - item_emdedding_distance_matrix = Pandas dataframe containing cosine distance matrix b/w items
    '''
    df_item_norm_sparse = sparse.csr_matrix(model.item_embeddings)
    similarities = cosine_similarity(df_item_norm_sparse)
    item_emdedding_distance_matrix = pd.DataFrame(similarities)
    item_emdedding_distance_matrix.columns = interactions.columns
    item_emdedding_distance_matrix.index = interactions.columns
    return item_emdedding_distance_matrix

def item_item_recommendation(item_emdedding_distance_matrix, item_id, 
                             item_dict, n_items = 10, show = True):
    '''
    Function to create item-item recommendation
    Required Input - 
        - item_emdedding_distance_matrix = Pandas dataframe containing cosine distance matrix b/w items
        - item_id  = item ID for which we need to generate recommended items
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - n_items = Number of items needed as an output
    Expected Output -
        - recommended_items = List of recommended items
    '''
    recommended_items = list(pd.Series(item_emdedding_distance_matrix.loc[item_id,:]. \
                                  sort_values(ascending = False).head(n_items+1). \
                                  index[1:n_items+1]))
    if show == True:
        print("Item of interest: {0}".format(item_dict[item_id]))
        print("Item similar to the above item:")
        counter = 1
        for i in recommended_items:
            print(str(counter) + '- ' +  item_dict[i])
            counter+=1
    return recommended_items

In [18]:
purchases = df[['order_id', 'product_id', 'GL_Margin']]

In [19]:
purchases.shape

(239938, 3)

In [20]:
from sklearn.preprocessing import LabelEncoder



In [21]:
purchases.sort_values("product_id")

,order_id,product_id,GL_Margin
648,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjyy5HmEQw,aa-2001,2.71
266,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchi8yLzkEQw,aa-2001,2.25
192302,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchiJz8W3Egw,aa-2001,7.46
1252,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjV24qgEgw,aa-2001,9.58
1518,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchi2rb6jEgw,aa-2001,8.44
1519,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchi2rb6jEgw,aa-2001,8.44
257,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchiqucrjEQw,aa-2001,4.81
2496,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchiVj6XBEgw,aa-2001,6.36
451,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchi9wrfjEQw,aa-2001,5.39
191338,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchidjfakEgw,aa-2001,6.76


In [22]:
new_prod = LabelEncoder()
purchases["prod_code"] = new_prod.fit_transform(purchases["product_id"])
purchases[["product_id", "prod_code"]].head(30)

,product_id,prod_code
0,hanes-5250,87
1,hanes-5250,87
2,hanes-5586,88
3,hanes-5250,87
4,hanes-5250,87
5,hanes-5250,87
6,hanes-5250,87
7,hanes-5250,87
8,hanes-5250,87
9,hanes-S04V,92


In [24]:
purchases2 = purchases[['order_id', 'prod_code', 'GL_Margin']]

In [25]:
purchases2.head()

,order_id,prod_code,GL_Margin
0,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjqo9foDgw,87,10.46
1,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchiooI_oDgw,87,13.48
2,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchi00Y7vDgw,88,11.47
3,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjStcDxDgw,87,13.48
4,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchiQpOz3Dgw,87,10.18


In [26]:
items = des[['prod_code', 'product_id']]

In [28]:
items.head()

,prod_code,product_id
0,0,aa-2001
1,1,aa-2102
2,2,aluminum-travel-mug-14
3,3,artprint-11x14
4,4,artprint-11x14-horizontal


In [30]:
interactions = create_interaction_matrix(df = purchases2,
                                         user_col = 'order_id',
                                         item_col = 'prod_code',
                                         rating_col = 'GL_Margin',
                                         threshold = '3')
interactions.shape



(199013, 146)

In [31]:
interactions.head()

prod_code,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,144,145
order_id,,,,,,,,,,,,,,,,,,,,,
OR_48PWYGT,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
OR_4S84P33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
OR_4V7TGD5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
OR_4VCNZVG,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
OR_4VG7934,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [32]:
user_dict = create_user_dict(interactions=interactions)


In [33]:
items_dict = create_item_dict(df = items,
                               id_col = 'prod_code',
                               name_col = 'product_id')

## Model

In [34]:
mf_model = runMF(interactions = interactions,
                 n_components = 30,
                 loss = 'warp',
                 k = '15',
                 epoch = 30,
                 n_jobs = 4)



## User Recommender

In [35]:
rec_list = sample_recommendation_user(model = mf_model, 
                                      interactions = interactions, 
                                      user_id = 'OR_855RQYK', 
                                      user_dict = user_dict,
                                      item_dict = items_dict, 
                                      threshold = 3,
                                      nrec_items = 10)

Known Products Bought:
1- hanes-5680
2- gildan-G500B
3- canvas-20x24-horizontal
4- canvas-16x24-horizontal
5- canvas-16x20-horizontal

 Recommended Products:
1- tote-13x13
2- canvas-32x48-horizontal
3- canvas-11x14
4- hanes-S04V
5- iphone-6
6- bella-8816
7- canvas-24x24
8- artprint-8x10
9- canvas-30x30
10- canvas-24x36-horizontal


## User - Item Recommender

In [36]:
sample_recommendation_item(model = mf_model,
                           interactions = interactions,
                           item_id = 1,
                           user_dict = user_dict,
                           item_dict = items_dict,
                           number_of_user = 15)

['ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchi_iqXpEQw',
 'ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjdhPaXDww',
 'ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjr4farEgw',
 'ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjIu-atEgw',
 'ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjB-dHkEQw',
 'ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchihxL-tEgw',
 'ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjvh9fiEQw',
 'ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjrluO_Egw',
 'ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjumJTqEQw',
 'OR_AAK5Y9E',
 'ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchinouStEgw',
 'ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjD97HmEQw',
 'ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchi-nYjsEQw',
 'ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchiwp-nlEQw',
 'OR_BKMNT78']

## Item Item Recommender

In [37]:
item_item_dist = create_item_emdedding_distance_matrix(model = mf_model,
                                                       interactions = interactions)

In [38]:
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                    item_id = 85,
                                    item_dict = items_dict,
                                    n_items = 10)



Item of interest: hanes-5250
Item similar to the above item:
1- hanes-5586
2- poster-24x36-horizontal
3- hanes-P1607
4- hanes-5680
5- pillow-indoor-18
6- canvas-8x10
7- iphone-7
8- iphone-6p
9- iphone-8-flexi
10- iphone-7pt
